In [1]:
%matplotlib qt5
from process_files import *
from func_science import *
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


# Flow of generating magnetograms
1. Load the config file corresponding to the data to be reduced
2. Get the darks data
3. Load meta data from the config file to derive structure of the data
4. Load flat field correction data
5. Load polarimetric modulation parameters (from the calibration data, stored in config file)
6. Load alignment parameters (from the target plate data, stored in config file)
7. Load the raw science data of interest and apply corrections (flat fielding, pol. demodulation, pol. crosstalk corrections) 
8. visualize the data --

--- yet to be done

9. Correct additional pol. crosstalks (e.g., instrumental polarization)
10. Combine the two beams
11. Derive magnetic field maps 

In [2]:
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
scdir = config['science']['directory']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# dark frames 
dk0 = data_cube(dkdir, line, 0, 0 )
dk0m = process_dark_cube(dk0.data)
dk1 = data_cube(dkdir, line, 1, 0)
dk1m = process_dark_cube(dk1.data)
dk2 = data_cube(dkdir, line, 2, 0)
dk2m = process_dark_cube(dk2.data)

Settings loaded from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/HELLRIDE_20230547_114744909_settings.ini
Data loading from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/Fe_I_6173/HELLRIDE_bbi_20230504_114756442_dk.DAT
Settings loaded from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/HELLRIDE_20230547_114744909_settings.ini
Data loading from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/Fe_I_6173/HELLRIDE_pol1_20230504_114756745_dk.DAT
Settings loaded from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/HELLRIDE_20230547_114744909_settings.ini
Data loading from  /arch/1/VTT/hellride/raw/20230504/Darks_1147/Fe_I_6173/HELLRIDE_pol2_20230504_114757033_dk.DAT


In [3]:
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wave_range = np.float64(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.float64(settings[linestr]['NWavePoints'])-1)
# Time stamps
tsfile = [f for f in os.listdir(scdir) if 'timestamp' in f]
tsfile = scdir + os.sep + tsfile[0]
ts = np.loadtxt(tsfile, delimiter=',')
im0ind, im1ind, im2ind = ts[:,3], ts[:,6], ts[:,9]
im0ind = im0ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
im1ind = im1ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
im2ind = im2ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
#
im0ind = im0ind - im0ind[:,0:1]
im1ind = im1ind - im1ind[:,0:1]
im2ind = im2ind - im2ind[:,0:1]
#
im0ind = im0ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im1ind = im1ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im2ind = im2ind.reshape([ncyc, nwav, nacc, nmod])%nmod

In [4]:
# bbi flats
ff0 = pf.open(config['flats'][line+'/bbi'])
ff0m = ff0[0].data[:,:,np.newaxis]
ff0m /= ff0m.mean()
ff0.close()
# pol1 flats
ff1 = pf.open(config['flats'][line+'/pol1'])
continuum1 = ff1[0].data
line_shifts1 = ff1[1].data
ff1m = ff1[2].data
ff1.close()
ff1msh = shift_3darray(ff1m/continuum1, line_shifts1)
ltempl1 = compute_mean_profile(ff1msh)
linearr1 = compute_shifted_lines(ltempl1, -line_shifts1)
ff1mast = ff1m/linearr1
# pol2 flats
ff2 = pf.open(config['flats'][line+'/pol2'])
continuum2 = ff2[0].data
line_shifts2 = ff2[1].data
ff2m = ff2[2].data
ff2.close()
ff2msh = shift_3darray(ff2m/continuum2, line_shifts2)
ltempl2 = compute_mean_profile(ff2msh)
linearr2 = compute_shifted_lines(ltempl2, -line_shifts2)
ff2mast = ff2m/linearr2

KeyError: 'Fe_I_6173/bbi'

In [5]:
plt.plot(ff2msh[600,600])
plt.plot(ff2msh[1000,1000])

In [6]:
# pol1 calib
pc1 = pf.open(config['pcalibration'][line+'/pol1'])
modmat1 = pc1[1].data
pc1.close()
demodmat1 = np.linalg.pinv(modmat1)
# pol2 calib
pc2 = pf.open(config['pcalibration'][line+'/pol2'])
modmat2 = pc2[1].data
pc2.close()
demodmat2 = np.linalg.pinv(modmat2)
# config params
rotang1 = np.float64(config['targetplate']['rotang1'])
rotang2 = np.float64(config['targetplate']['rotang2'])
mag = np.float64(config['targetplate']['mag'])
shift1_y = np.float64(config['targetplate']['shift1_y'])
shift1_x = np.float64(config['targetplate']['shift1_x'])
shift2_y = np.float64(config['targetplate']['shift2_y'])
shift2_x = np.float64(config['targetplate']['shift2_x'])
bbi_scale = np.float64(config['targetplate']['bbi_scale'])
pol1_scale = mag*bbi_scale
pol2_scale = mag*bbi_scale

In [7]:
# Get science data; test it only on one set
for i in range(1):
    dc0 = data_cube(scdir, line, 0, i)
    Y = X = dc0.roi
    sc0 = dc0.data - dk0m
    sc0corr = sc0/ff0m
    #
    dc1 = data_cube(scdir, line, 1, i)
    sc1 = dc1.data.reshape([Y,X,nacc*nmod*nwav], order='F') - dk1m
    sc1_add = coadd_modulated_imgs (sc1, im1ind[i], nmod, nacc, nwav)
    sc1_ff = sc1_add/ff1mast[:,:,np.newaxis,:]
    sc1_sh = shift_lines_4d(sc1_ff, line_shifts1)
    sc1_demod = np.einsum('ijkl,mk->ijml', sc1_sh, demodmat1)
    sc1_demod[:,:,1::,:] /= sc1_demod[:,:,0,:][:,:,np.newaxis,:]
    sc1_demod = np.fliplr(sc1_demod)
    sc1_i2quv_corr = correct_i2quv(sc1_demod)
    #
    dc2 = data_cube(scdir, line, 2, i)
    sc2 = dc2.data.reshape([Y,X,nacc*nmod*nwav], order='F') - dk2m
    sc2_add = coadd_modulated_imgs (sc2, im2ind[i], nmod, nacc, nwav)
    sc2_ff = sc2_add/ff2mast[:,:,np.newaxis,:]
    sc2_sh = shift_lines_4d(sc2_ff, line_shifts2)
    sc2_demod = np.einsum('ijkl,mk->ijml', sc2_sh, demodmat2)
    sc2_demod[:,:,1::,:] /= sc2_demod[:,:,0,:][:,:,np.newaxis,:] 
    sc2_demod = np.flipud(np.fliplr(sc2_demod))
    sc2_i2quv_corr = correct_i2quv(sc2_demod)

Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_bbi_20220302_131732198_sc.DAT
Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_pol1_20220302_131732544_sc.DAT


100%|██████████| 1280/1280 [01:48<00:00, 11.78it/s]


I-->Q crosstalk distribution (mean, stdev): 0.0222 0.0062
I-->U crosstalk distribution (mean, stdev): -0.0357 0.0101
I-->V crosstalk distribution (mean, stdev): -0.1761 0.0063
Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_pol2_20220302_131732905_sc.DAT


100%|██████████| 1280/1280 [01:47<00:00, 11.90it/s]


I-->Q crosstalk distribution (mean, stdev): 0.0357 0.008
I-->U crosstalk distribution (mean, stdev): -0.1056 0.0143
I-->V crosstalk distribution (mean, stdev): -0.172 0.0056


In [8]:
# sc1_ff = sc1_add/ff1mast[:,:,np.newaxis,:]
plt.imshow(sc2_i2quv_corr[100:-100,100:-100,0,5], cmap='gray')

In [9]:
def plot_stokes_figures(data_4d, npts, pscale_im, pscale_wav, lcore=5, lwing=4):
    """
    Plot stokes images and profiles at selected points of the image
    Input:  4d spectro-polarimetric imaging data
            number of points to be plotted
            (opt.) line core position in pixels
            (opt.) line wing width in pixels
    Output: figure showing stokes images and profiles
    """
    plt.figure()
    plt.imshow(data_4d[:,:,0,0])
    pts = plt.ginput(npts)
    plt.close()
    #
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    circ_size=20*pscale_im
    ny, nx, nm, nw = data_4d.shape
    #
    fig, axs = plt.subplots(nrows=6, ncols=2, figsize=(8,11))
    gs = axs[0,0].get_gridspec()
    for ax in axs[0:4,:]:
        for ax_ in ax:
            ax_.remove()
    axii = fig.add_subplot(gs[0:2,0])
    axqq = fig.add_subplot(gs[0:2,1])
    axuu = fig.add_subplot(gs[2:4,0])
    axvv = fig.add_subplot(gs[2:4,1])
    axi, axq, axu, axv = axs[4,0], axs[4,1], axs[5,0], axs[5,1]
    im_extent = [0,nx*pscale_im,0,ny*pscale_im]
    # print(im_extent)
    waves = (np.arange(nw)-5)*pscale_wav
    #
    axci = axii.imshow(data_4d[:,:,0,lcore], extent=im_extent, cmap='gray')
    axcq = axqq.imshow(np.mean(data_4d[:,:,1,lcore-lwing:lcore],axis=2), extent=im_extent, cmap='gray')
    axcu = axuu.imshow(np.mean(data_4d[:,:,2,lcore-lwing:lcore],axis=2), extent=im_extent, cmap='gray')
    axcv = axvv.imshow(np.mean (data_4d[:,:,3,lcore-lwing:lcore],axis=2), extent=im_extent, cmap='gray')
    #
    fig.colorbar(axci, ax=axii, fraction=0.0475, pad=0.0125)
    fig.colorbar(axcq, ax=axqq, fraction=0.0475, pad=0.0125)
    fig.colorbar(axcu, ax=axuu, fraction=0.0475, pad=0.0125)
    fig.colorbar(axcv, ax=axvv, fraction=0.0475, pad=0.0125)

    #
    axi.set_ylabel('I (arbitrary units)')
    axq.set_ylabel('Q/I')
    axu.set_ylabel('U/I')
    axv.set_ylabel('V/I')
    for ax in [axi, axq, axu, axv]:
        ax.set_xlabel('Wavelength from line core (in $\AA$)')
    #
    axii.set_title('Stokes-I map (core)')
    axqq.set_title('Stokes-Q map (wing)')
    axuu.set_title('Stokes-U map (wing)')
    axvv.set_title('Stokes-V map (wing)')
    #
    for ax in [axii, axqq, axuu, axvv]:
            ax.set_xlabel('arcsec')
            ax.set_ylabel('arcsec')
    #
    for i, xy in enumerate(pts):
        x, y = xy
        xa, ya = x*pscale_im, (ny-y)*pscale_im
        axii.add_patch(plt.Circle((xa,ya),circ_size,fill=False,color=colors[i]))
        axqq.add_patch(plt.Circle((xa,ya),circ_size,fill=False,color=colors[i]))
        axuu.add_patch(plt.Circle((xa,ya),circ_size,fill=False,color=colors[i]))
        axvv.add_patch(plt.Circle((xa,ya),circ_size,fill=False,color=colors[i]))
        axi.plot(waves, data_4d[int(y),int(x),0])
        axq.plot(waves, data_4d[int(y),int(x),1])
        axu.plot(waves, data_4d[int(y),int(x),2])
        axv.plot(waves, data_4d[int(y),int(x),3])
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.35, hspace=0.4)

In [11]:
crop = 100
plot_stokes_figures(sc2_i2quv_corr[crop:-crop,crop:-crop], 3, pol2_scale, wave_step)
# plt.savefig('stokes_map.png', dpi=600)

In [36]:
from matplotlib import animation
vid_data = sc2_i2quv_corr[crop:-crop, crop:-crop,0,:]
i, vid_size = 0, 10
N = vid_data.shape[2]
DPI = vid_data.shape[0]/vid_size
#
fig, ax = plt.subplots(1,1,figsize=(10,10))
disp = ax.imshow(vid_data[:,:,0], animated=True)
def update_frame(i):
    disp.set_array(vid_data[:,:,i])
    return disp,
writer = animation.writers['ffmpeg'](fps=30)
ani = animation.FuncAnimation(fig, update_frame,  blit=True)
ani.save('sc2_i2quv_corr_I.mp4', writer=writer, dpi=108)
plt.show()

IndexError: index 15 is out of bounds for axis 2 with size 15

In [31]:
plt.close()

In [15]:
vid_data.shape

(1080, 1080, 15)